# LBG Senior Data Scientist - CDAO

## EDA

In [481]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests
import re
import geocoder

from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.metrics import classification_report, precision_score, recall_score, roc_curve, roc_auc_score, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from typing import Iterable
from datetime import date

%matplotlib inline

In [385]:
# Read DFs
mortgages_df = pd.read_csv('../data/mortgage.csv', parse_dates=[1])
campaign_df = pd.read_csv('../data/campaign.csv')

In [386]:
campaign_df.head()

,participant_id,name_title,first_name,last_name,age,postcode,marital_status,education,job_title,occupation_level,education_num,familiarity_FB,view_FB,interested_insurance,company_email,created_account
0,8.997000e+12,Mr.,Dale,Coles,39,EH4 9FL,Never-married,Bachelors,Furniture conservator/restorer,1,17,7,9,0,jrhodes@jones.com,No
1,2.764970e+12,NaN,Joel,Allen,50,LS5H 0JG,Married-civ-spouse,Bachelors,"Administrator, sports",4,17,9,6,1,william32@carey.com,No
2,8.454880e+12,Mr.,Craig,Davis,38,EH8W 2QJ,Divorced,HS-grad,Armed forces training and education officer,12,12,5,4,1,smithvanessa@coles.org,No
3,5.781320e+12,Mr.,Brandon,Thornton,53,EH2 5UE,Married-civ-spouse,11th,Facilities assistant,1,9,9,2,0,sbarrett@hart.biz,No
4,8.687490e+12,Miss,Brett,Fletcher,28,SN19 0PU,Married-civ-spouse,Bachelors,Information systems manager,12,17,8,9,1,lparkinson@butler.com,No


In [387]:
mortgages_df.head()

,full_name,dob,town,paye,salary_band,years_with_employer,months_with_employer,hours_per_week,capital_gain,capital_loss,new_mortgage,sex,religion,relationship,race,native_country,workclass,demographic_characteristic
0,Mr. Dale Coles,1979-05-10,Edinburgh,53791313,£18109 yearly,20,6,40,2174,0,Yes,Male,Christianity,Not-in-family,White,United Kingdom,State-gov,77516
1,Joel Allen,1968-07-06,Leeds,24005258,£16945 yearly,28,1,13,0,0,Yes,Male,Christianity,Husband,White,United Kingdom,Self-emp-not-inc,83311
2,Mr. Craig Marc Davis,1979-02-10,Edinburgh,44124649,£729.87 pw,14,5,40,0,0,Yes,Male,Christianity,Not-in-family,White,United Kingdom,Private,215646
3,Mr. Brandon Thornton,1965-10-20,Edinburgh,85701281,£19087 yearly,32,6,40,0,0,Yes,Male,Christianity,Husband,Black,United Kingdom,Private,234721
4,Miss Brett Carol Fletcher,1990-02-20,Swindon,BR442000,£2741.5 per month,3,6,40,0,0,Yes,Female,Christianity,Wife,Black,Sweden,Private,338409


In [388]:
mortgages_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   full_name                   32561 non-null  object        
 1   dob                         32561 non-null  datetime64[ns]
 2   town                        32561 non-null  object        
 3   paye                        32561 non-null  object        
 4   salary_band                 32561 non-null  object        
 5   years_with_employer         32561 non-null  int64         
 6   months_with_employer        32561 non-null  int64         
 7   hours_per_week              32561 non-null  int64         
 8   capital_gain                32561 non-null  int64         
 9   capital_loss                32561 non-null  int64         
 10  new_mortgage                32561 non-null  object        
 11  sex                         32561 non-null  object    

In [389]:
# Split out the last name for mortgages
mortgages_df['last_name'] = mortgages_df.full_name.str.rsplit(n=1, expand=True)[1]
mortgages_df.last_name

0            Coles
1            Allen
2            Davis
3         Thornton
4         Fletcher
           ...    
32556     Williams
32557        Moore
32558    Alexander
32559         Mann
32560        Green
Name: last_name, Length: 32561, dtype: object

In [390]:
def get_name_title(full_name):
    split = full_name.split()
    if split[0] not in ['Mr.', 'Miss', 'Dr.', 'Ms.', 'Mrs.']:
        return 'None'
    else:
        return split[0]

In [391]:
def get_first_name(full_name):
    split = full_name.split()
    if split[0] in ['Mr.', 'Miss', 'Dr.', 'Ms.', 'Mrs.']:
        return split[1]
    else:
        return split[0]

In [392]:
mortgages_df['name_title'] = mortgages_df.full_name.apply(get_name_title)
mortgages_df['first_name'] = mortgages_df.full_name.apply(get_first_name)

In [393]:
mortgages_df[['full_name', 'name_title', 'first_name', 'last_name']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   full_name   32561 non-null  object
 1   name_title  32561 non-null  object
 2   first_name  32561 non-null  object
 3   last_name   32561 non-null  object
dtypes: object(4)
memory usage: 1017.7+ KB


In [394]:
campaign_df['name_title'] = campaign_df.name_title.fillna('None')
campaign_df[['name_title', 'first_name', 'last_name']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32060 entries, 0 to 32059
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name_title  32060 non-null  object
 1   first_name  32060 non-null  object
 2   last_name   32060 non-null  object
dtypes: object(3)
memory usage: 751.5+ KB


In [395]:
# Calculate age from date of birth
def calculate_age(dob, current_year, current_month, current_day):
    today = date(current_year, current_month, current_day)
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    return age

In [396]:
# for year in [2017, 2018, 2019]:
#     for month in range(1, 13):
#         for day in range(1, 32):
#             try:
#                 mortgages_df['age'] = mortgages_df.dob.apply(lambda x: calculate_age(x, year, month, day))
                
#                 pd.testing.assert_series_equal(campaign_df.age[:3], mortgages_df.age[:3])
#             except AssertionError:
#                 continue
#             except ValueError:
#                 continue
#             print(f"Successfully matched date {year, month, day}")
#             break

In [397]:
# Get postcode areas
postcode_areas = pd.read_csv("../data/areas.csv")[['Postcode area', 'Area covered']]
postcode_areas.columns = ['area', 'town']

In [398]:
postcode_areas

,area,town
0,AB,Aberdeen
1,AL,St Albans
2,B,Birmingham
3,BA,Bath
4,BB,Blackburn
...,...,...
120,YO,York
121,ZE,Lerwick
122,GY,"""Guernsey"""
123,JE,"""Jersey"""


In [399]:
campaign_df['area'] = campaign_df.postcode.str.split('\d', expand=True)[0]
campaign_df = pd.merge(left=campaign_df, left_on='area', right=postcode_areas, right_on='area', how='left')

In [443]:
combined_df = pd.merge(
    left=campaign_df, left_on=['last_name', 'first_name', 'name_title', 'town'],
    right=mortgages_df, right_on=['last_name', 'first_name', 'name_title', 'town'],
    how='inner'
)

In [444]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27031 entries, 0 to 27030
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   participant_id              27031 non-null  float64       
 1   name_title                  27031 non-null  object        
 2   first_name                  27031 non-null  object        
 3   last_name                   27031 non-null  object        
 4   age                         27031 non-null  int64         
 5   postcode                    27031 non-null  object        
 6   marital_status              27031 non-null  object        
 7   education                   27031 non-null  object        
 8   job_title                   27031 non-null  object        
 9   occupation_level            27031 non-null  int64         
 10  education_num               27031 non-null  int64         
 11  familiarity_FB              27031 non-null  int64     

In [445]:
# Clean up - no longer needed
unneeded_cols = [
    'participant_id', 'name_title', 'first_name', 'last_name', 'postcode',
    'area', 'full_name', 'dob', 'paye', 'new_mortgage', 'company_email'
]
combined_df = combined_df.drop(unneeded_cols, axis=1)

In [446]:
# Clean up - duplicate columns
dupe_cols = ['education', 'job_title', 'relationship']
combined_df = combined_df.drop(dupe_cols, axis=1)

In [447]:
combined_df.columns

Index(['age', 'marital_status', 'occupation_level', 'education_num',
       'familiarity_FB', 'view_FB', 'interested_insurance', 'created_account',
       'town', 'salary_band', 'years_with_employer', 'months_with_employer',
       'hours_per_week', 'capital_gain', 'capital_loss', 'sex', 'religion',
       'race', 'native_country', 'workclass', 'demographic_characteristic'],
      dtype='object')

In [448]:
# Clean up - fairness colums - come back to later
fairness_cols = ['sex', 'religion', 'race', 'native_country', 'town']
combined_df = combined_df.drop(fairness_cols, axis=1)

In [450]:
# Features we can use for either model
campaign_features = ['age', 'marital_status', 'occupation_level', 'education_num', 'familiarity_FB', 'view_FB', 'interested_insurance', 'created_account']
mortgage_features = ['salary_band', 'years_with_employer', 'months_with_employer', 'hours_per_week', 'capital_gain', 'capital_loss', 'workclass', 'demographic_characteristic']

In [459]:
campaign_model_df = combined_df[campaign_features]
mortage_model_df = combined_df[mortgage_features]

In [460]:
campaign_model_df

,age,marital_status,occupation_level,education_num,familiarity_FB,view_FB,interested_insurance,created_account
0,39,Never-married,1,17,7,9,0,No
1,50,Married-civ-spouse,4,17,9,6,1,No
2,38,Divorced,12,12,5,4,1,No
3,53,Married-civ-spouse,1,9,9,2,0,No
4,28,Married-civ-spouse,12,17,8,9,1,No
...,...,...,...,...,...,...,...,...
27026,23,Never-married,8,17,9,5,0,NaN
27027,34,Never-married,4,12,7,4,1,NaN
27028,44,Married-civ-spouse,6,17,3,4,1,NaN
27029,60,Widowed,6,13,6,2,1,NaN


In [463]:
# Map Yes/No to binary outcomes
map_dict = {"Yes": 1, "No": 0}
campaign_model_df['created_account'] = campaign_model_df.created_account.map(map_dict)

C:\Users\josep\AppData\Local\Temp\ipykernel_13304\4235329814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_model_df['created_account'] = campaign_model_df.created_account.map(map_dict)


In [479]:
# Split the data into labelled and unlabelled sets
X_labelled = pd.get_dummies(campaign_model_df[~campaign_model_df.created_account.isna()].drop('created_account', axis=1))
X_unlabelled = pd.get_dummies(campaign_model_df[campaign_model_df.created_account.isna()].drop('created_account', axis=1))
y = campaign_model_df[~campaign_model_df.created_account.isna()].created_account

In [482]:
# Test performance of random forest 
random_forest = RandomForestClassifier()
rf_score = cross_val_score(random_forest, X_labelled, y, scoring='f1').mean()
rf_score

0.31470584714824346

In [484]:
# Look at most important features
random_forest.fit(X_labelled, y)
feature_importances = random_forest.feature_importances_
feature_names = X_labelled.columns
pd.DataFrame(zip(feature_names, feature_importances)).sort_values(by=1, ascending=False)

,0,1
1,occupation_level,2.454456e-01
0,age,2.077472e-01
2,education_num,1.524364e-01
4,view_FB,1.518415e-01
3,familiarity_FB,1.355402e-01
8,marital_status_Married-civ-spouse,4.789104e-02
5,interested_insurance,3.184419e-02
10,marital_status_Never-married,1.533909e-02
6,marital_status_Divorced,6.623965e-03
11,marital_status_Separated,2.046203e-03
